# GAN exercise- advanced
Read the instructions in the attached pdf.
Good luck!

In [2]:
import tensorflow as tf
import sys
import numpy as np
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [32]:
from os import listdir, path
from keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm 

input_size=(64, 64, 3)
data_train = np.zeros(shape=(13233, 64, 64, 3))
im_idx = 0

for folder in tqdm(listdir('dataset/dataset')):
    for pic in listdir('dataset/dataset/' + folder):
        if pic[-2:] == 'db':
            continue

        image = load_img(path.join('dataset/dataset', folder, pic), target_size=input_size)
        image = img_to_array(image)
        data_train[im_idx, :, :, :] = image

        im_idx += 1

print(im_idx)

100%|█████████████████████████████████████████████████████████████████████████████| 5749/5749 [00:43<00:00, 131.06it/s]

13233


In [33]:
data_train = data_train.astype('float32')
data_train = data_train / 255.0

In [76]:
type(data_train)

numpy.ndarray

In [77]:
np.save('data_train', data_train)

In [12]:
from tensorflow.keras import activations
from tensorflow.keras.layers import MaxPooling2D, Activation, Dropout
from keras.layers.convolutional import Conv2D

class Block_Discriminator(tf.keras.Model):
    def __init__(self):
        super(Block_Discriminator, self).__init__()
        
        self.conv = Conv2D(filters=3, kernel_size=(3, 3))
        self.max_pooling = MaxPooling2D()
        self.relu = Activation(activations.relu)
        self.dropout = Dropout(rate=0.2)

    def call(self, x):
        x = self.conv(x)
        x = self.max_pooling(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        return x

In [43]:
from tensorflow.keras.layers import Flatten, Dense

class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.block1 = Block_Discriminator()
        self.block2 = Block_Discriminator()
        self.block3 = Block_Discriminator()
        
        self.flatten = Flatten()
        
        self.dense1 = Dense(units=256, activation='tanh')
        self.dropout1 = Dropout(rate=0.2)
        self.dense2 = Dense(units=126, activation='tanh')
        self.dropout2 = Dropout(rate=0.2)
        self.dense3 = Dense(units=1, activation='sigmoid')
    
    def call(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        
        x = self.flatten(x)
        
        x = self.dense1(x)
        x = self.dropout1(x)
        x = self.dense2(x)
        x = self.dropout2(x)
        x = self.dense3(x)
        
        return x

In [44]:
from keras.optimizers import Adam

discriminator = Discriminator()
discriminator.build(input_shape=(None, 64, 64, 3))
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), 
                      metrics=['accuracy'])

discriminator.summary()

Model: "discriminator_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block__discriminator_19 (Blo multiple                  84        
_________________________________________________________________
block__discriminator_20 (Blo multiple                  84        
_________________________________________________________________
block__discriminator_21 (Blo multiple                  84        
_________________________________________________________________
flatten_6 (Flatten)          multiple                  0         
_________________________________________________________________
dense_18 (Dense)             multiple                  27904     
_________________________________________________________________
dropout_33 (Dropout)         multiple                  0         
_________________________________________________________________
dense_19 (Dense)             multiple              

In [54]:
from numpy.random import randint, rand

def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix, :, :, :]
    y = np.ones((n_samples, 1))
    return X, y

def generate_fake_samples(n_samples):
    X = rand(64 * 64 * 3 * n_samples)
    X = X.reshape((n_samples, 64, 64, 3))
    y = np.zeros((n_samples, 1))
    
    return X, y

In [55]:
def train_discriminator(model, dataset, n_iter=100, n_batch=256):
    half_batch = int(n_batch / 2)
    for i in range(n_iter):
        X_real, y_real = generate_real_samples(dataset, half_batch)
        _, real_acc = model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(half_batch)
        _, fake_acc = model.train_on_batch(X_fake, y_fake)
        print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))

In [56]:
train_discriminator(discriminator, data_train)

>1 real=98% fake=0%
>2 real=100% fake=2%
>3 real=97% fake=5%
>4 real=91% fake=9%
>5 real=73% fake=33%
>6 real=58% fake=56%
>7 real=55% fake=75%
>8 real=43% fake=79%
>9 real=47% fake=88%
>10 real=38% fake=92%
>11 real=24% fake=87%
>12 real=22% fake=93%
>13 real=22% fake=94%
>14 real=23% fake=99%
>15 real=32% fake=96%
>16 real=31% fake=98%
>17 real=39% fake=97%
>18 real=41% fake=99%
>19 real=40% fake=99%
>20 real=47% fake=97%
>21 real=47% fake=99%
>22 real=44% fake=99%
>23 real=53% fake=98%
>24 real=75% fake=99%
>25 real=69% fake=100%
>26 real=71% fake=98%
>27 real=73% fake=99%
>28 real=75% fake=99%
>29 real=80% fake=99%
>30 real=84% fake=100%
>31 real=88% fake=100%
>32 real=88% fake=98%
>33 real=91% fake=100%
>34 real=95% fake=100%
>35 real=92% fake=99%
>36 real=96% fake=100%
>37 real=97% fake=100%
>38 real=93% fake=98%
>39 real=98% fake=100%
>40 real=100% fake=99%
>41 real=98% fake=99%
>42 real=98% fake=100%
>43 real=99% fake=100%
>44 real=99% fake=100%
>45 real=98% fake=99%
>46 real=9

In [61]:
from tensorflow.keras.layers import UpSampling2D, Conv2DTranspose, BatchNormalization

class Block_Generator(tf.keras.Model):
    def __init__(self):
        super(Block_Generator, self).__init__()
        
        self.up_sampling = UpSampling2D(size=(3, 3))
        self.max_pooling = Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), 
                                           padding='same')
        self.relu = Activation(activations.relu)
        self.dropout = Dropout(rate=0.2)
        self.batch_norm = BatchNormalization()

    def call(self, x):
        x = self.up_sampling(x)
        x = self.max_pooling(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)
        
        return x

In [67]:
from tensorflow.keras.layers import Reshape

class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
                
        self.dense1 = Dense(units=64*7*7, activation='tanh')
        self.dropout1 = Dropout(rate=0.2)
        self.reshape = Reshape((7, 7, 64))
        
        self.block1 = Block_Generator()
        self.block2 = Block_Generator()
        self.block3 = Block_Generator()
    
    def call(self, x):
        x = self.dense1(x)
        x = self.dropout1(x)
        x = self.reshape(x)
        
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        
        return x

In [68]:
generator = Generator()

In [69]:
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y

In [71]:
from keras.models import Sequential

discriminator.trainable = False

gan_model = Sequential()
gan_model.add(generator)
gan_model.add(discriminator)

opt = Adam(lr=0.0002, beta_1=0.5)
gan_model.compile(loss='binary_crossentropy', optimizer=opt)

In [73]:
import matplotlib.pyplot as plt

def save_plot(examples, epoch, n=10):
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')
        filename = 'generated_plot_e%03d.png' % (epoch+1)
        plt.savefig(filename)
        plt.close()

In [74]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    X_real, y_real = generate_real_samples(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    
    save_plot(x_fake, epoch)
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)

In [75]:
n_batch = 256
n_epochs = 100
latent_dim = 100
dataset = data_train
batch_per_epoch = int(dataset.shape[0] / n_batch)
half_batch = int(n_batch / 2)

for i in range(n_epochs):
    for j in range(batch_per_epoch):
        X_real, y_real = generate_real_samples(dataset, half_batch)
        X_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)
        X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
        d_loss, _ = discriminator.train_on_batch(X, y)
        X_gan = generate_latent_points(latent_dim, n_batch)
        y_gan = ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        
        print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
    if (i+1) % 10 == 0:
        summarize_performance(i, generator, discriminator, dataset, latent_dim)

ResourceExhaustedError:  OOM when allocating tensor with shape[32,1512,1512,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node generator_1/block__generator_5/conv2d_transpose_5/conv2d_transpose (defined at <ipython-input-61-8444659bfc44>:16) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_predict_function_6167]

Errors may have originated from an input operation.
Input Source operations connected to node generator_1/block__generator_5/conv2d_transpose_5/conv2d_transpose:
 generator_1/block__generator_5/up_sampling2d_5/resize/ResizeNearestNeighbor (defined at <ipython-input-61-8444659bfc44>:15)

Function call stack:
predict_function
